In [1]:
# Installing confluent-kafka
!pip install confluent-kafka==1.7.0

     |████████████████████████████████| 2.7 MB 3.1 MB/s eta 0:00:01


In [2]:
# Importing libraries

import sys
import time
import pandas as pd
import json

from confluent_kafka.admin import AdminClient, NewTopic, NewPartitions
from confluent_kafka import KafkaException
from confluent_kafka import SerializingProducer
from confluent_kafka.serialization import *
from uuid import uuid4
from IPython.display import clear_output

In [3]:
# Importing data (to simulate the readings of a sensor)
Univariate_data = pd.read_csv('Univariate_Labeled.csv', parse_dates=['DateTime'], index_col=['DateTime'])
Univariate_data = Univariate_data[0:34944] # 52 weeks
Univariate_data

,Consumption,Anomaly
DateTime,,
1997-01-01 00:00:00,950.0,0
1997-01-01 00:15:00,939.0,0
1997-01-01 00:30:00,943.0,0
1997-01-01 00:45:00,971.0,0
1997-01-01 01:00:00,1014.0,0
...,...,...
1997-12-30 22:45:00,886.0,0
1997-12-30 23:00:00,883.0,0
1997-12-30 23:15:00,868.0,0


In [4]:
# Producer's configuration

producer_conf = {
        'bootstrap.servers': "kafka:9092",
        'key.serializer': StringSerializer('utf_8'),
        'value.serializer': StringSerializer('utf_8')
}

producer = SerializingProducer(producer_conf)

In [5]:
# Delivery report message
def delivery_report(err, msg):
    if err is not None:
        print("Failed to deliver message: {}".format(err))
    else:
        print("Produced record to topic {} partition [{}] @ offset {}".format(msg.topic(), msg.partition(), msg.offset()))

In [6]:
# Sending messages

# for i in range(Univariate_data.shape[0]): # Run this
for i in range(40): # Debug
    key = "EC1"
    value = {"ElectricityConsumption": Univariate_data.iloc(0)[i][0], "ts":int(time.time())}
    producer.produce(topic="Elec_Cons_Topic", value=json.dumps(value), key=key, on_delivery=delivery_report)
    print(value)
    producer.poll(1)
    time.sleep(10)

{'ElectricityConsumption': 950.0, 'ts': 1667227180}
{'ElectricityConsumption': 939.0, 'ts': 1667227191}
Produced record to topic Elec_Cons_Topic partition [0] @ offset 0
{'ElectricityConsumption': 943.0, 'ts': 1667227201}
Produced record to topic Elec_Cons_Topic partition [0] @ offset 1
{'ElectricityConsumption': 971.0, 'ts': 1667227211}
Produced record to topic Elec_Cons_Topic partition [0] @ offset 2
{'ElectricityConsumption': 1014.0, 'ts': 1667227221}
Produced record to topic Elec_Cons_Topic partition [0] @ offset 3
{'ElectricityConsumption': 1041.0, 'ts': 1667227231}
Produced record to topic Elec_Cons_Topic partition [0] @ offset 4
{'ElectricityConsumption': 1023.0, 'ts': 1667227241}
Produced record to topic Elec_Cons_Topic partition [0] @ offset 5
{'ElectricityConsumption': 1030.0, 'ts': 1667227251}
Produced record to topic Elec_Cons_Topic partition [0] @ offset 6
{'ElectricityConsumption': 1004.0, 'ts': 1667227261}
Produced record to topic Elec_Cons_Topic partition [0] @ offset 7